In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import sys
import glob
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Clean Data

In [21]:
all_files = glob.glob('../data/*.csv')
all_files = sorted(all_files)
df = pd.concat([pd.read_csv(files) for files in all_files])
df = df.iloc[:, 0:22]

In [49]:
df.sort_values('impressions', ascending=False).head(10)

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,user profile clicks,url clicks,hashtag clicks,detail expands,permalink clicks,app opens,app installs,follows,email tweet,dial phone,media views,media engagements
27,1356438150915125250,https://twitter.com/420_note/status/1356438150915125250,今日からオレゴン州で全ての薬物の所持が正式に非犯罪化されました。\n\n下記の量を所持してても刑事罰はありません。\n\n- ヘロイン1g未満\n- MDMAの1g未満または5ピル未満\n- メタンフェタミン2g未満\n- LSDが40枚未満\n- シロシビン12g未満\n- コカイン2g未満\n\nhttps://t.co/NNX5eYZiGv,2021-02-02 03:03 +0000,266019.0,16831.0,0.063270,528.0,6.0,1315.0,1216.0,1595.0,0.0,12112.0,0.0,0,0,2,0,0,57,57
68,1382714917204086793,https://twitter.com/420_note/status/1382714917204086793,@gjmorley NYが嗜好用大麻を合法化した事で、アメリカ在住日本人の80%以上は嗜好大麻が合法な地域で生活してることになりました。医療大麻ではそれが90%。\n\nデータは15ある日本の大使館・総領事館・領事事務所に在留届を出してる外務省の統計を元にしています。\n\n是非参考にして下さい。\nhttps://t.co/Y63JbXeFPB https://t.co/i1wzYy7gNY,2021-04-15 15:18 +0000,119087.0,1689.0,0.014183,25.0,1.0,57.0,61.0,35.0,0.0,527.0,0.0,0,0,0,0,0,983,983
64,1334134242473771008,https://twitter.com/420_note/status/1334134242473771008,今日の国連の採決で大麻を最も厳しいSchedule IVからSchedule I へ可決されましたが、日本は反対しています。\n\nそれ以外に、大麻チンキをSchedule Iからの除外や、THCが0.2%未満の大麻を麻薬条約から除外する内容など、4つの採決で全て日本は反対しています。。。\n\n詳細は↓\n\nhttps://t.co/PdNlB8NbYp,2020-12-02 13:56 +0000,110146.0,2697.0,0.024486,61.0,4.0,176.0,228.0,883.0,0.0,1345.0,0.0,0,0,0,0,0,0,0
2,1300038336539623424,https://twitter.com/420_note/status/1300038336539623424,@tenchim_1119 アメリカでは医療目的以外にも人権問題として大麻の規制緩和がなされてます。この際、是非大麻にまつわる社会情勢を知って頂ければ幸いです。\n\nhttps://t.co/DVyyq6auLh,2020-08-30 11:51 +0000,102446.0,946.0,0.009234,4.0,1.0,13.0,188.0,131.0,0.0,609.0,0.0,0,0,0,0,0,0,0
3,1387926835829252097,https://twitter.com/420_note/status/1387926835829252097,"カナダで乾燥大麻を使ったvape をテストするための人員を募集してて、それが年収 50,000カナダドル（約443万円）です。\n\n年間3週間の有給付きで、乾燥大麻を購入すために毎月 250カナダドル（約2.2万円）が給料とは別に支給されます。\n\nhttps://t.co/vHBG3Qgqz7 https://t.co/UBkzFytQyR",2021-04-30 00:28 +0000,86825.0,6329.0,0.072894,178.0,4.0,602.0,258.0,961.0,0.0,2584.0,0.0,0,0,0,0,0,1742,1742
44,1384300783156350986,https://twitter.com/420_note/status/1384300783156350986,幸せな生活を送る事ができるのであれば基本全ての薬物(カフェイン、酒、大麻等)は一切推奨しません。\n\nしかし、アメリカ在住日本人の80~90%が嗜好・医療大麻合法州で生活し、アメリカ人の91%が嗜好・医療で大麻が合法化されるべきだと思ってる事を考えると #大麻吸ってみたい と思ってもおかしく無い。 https://t.co/uT0iTsIRfm,2021-04-20 00:20 +0000,86219.0,3090.0,0.035839,36.0,0.0,113.0,149.0,28.0,114.0,195.0,0.0,0,0,0,0,0,2455,2455
2,1189450399729647617,https://twitter.com/420_note/status/1189450399729647617,イスラエルの会社が大麻とマジックマッシュルームを組み合わせて、「現在の医薬品では不十分に対処されている病気」の治療の研究をすることを発表しました！\n両方ともアメリカでSchedule 1 に指定されている物を、このような形で研究ができるイスラエルはすごいの一言！\n\nhttps://t.co/X8q1tXkt7n,2019-10-30 07:54 +0000,79998.0,2066.0,0.025826,202.0,6.0,687.0,298.0,488.0,0.0,384.0,0.0,0,0,1,0,0,0,0
4,1289007689368928257,https://twitter.com/420_note/status/1289007689368928257,速報！\n\n嗜好大麻を合法化してる州に対して予算を使って取り締まる事を禁止する案に、下院議会が通りました！問題は共和党の上院ですが「小さな政府」が根本思想にあるので是非通して欲しい。\n\n連邦議会が大麻の規制緩和に賛成したのは、昨年のSTATES Actに続き、史上2回目。\n https://t.co/7TeqDdgIlW,2020-07-31 01:19 +0000,76450.0,1291.0,0.016887,44.0,2.0,131.0,159.0,86.0,0.0,869.0,0.0,0,0,0,0,0,0,0
16,1405686542706765825,https://twitter.com/420_note/status/1405686542706765825,人口356万人のConnecticutが19州目の嗜好大麻合法州になりました👏㊗️\n\n来月から外出時に42gの所持、自宅・社内では141gの所持、6株までの栽培が合法になり、2022年5月から販売が開始されます。\n\nNJが去年合法してから、NY・CTのトライステイトの動きが早い！\nhttps://t.co/cukxc1smxC,2021-06-18 00:39 +0000,72998.0,1376.0,0.018850,110.0,3.0,316.0,126.0,109.0,0.0,712.0,0.0,0,0,0,0,0,0,0
52,1195917841725186050,https://twitter.com/420_note/status/1195917841725186050,キターーー！！\n\n大麻をScheduleの規制薬物から外し（=連邦法の管理下に置かないので実質「合法化」）、薬物戦争の被害者のための支援金を確保し、大麻が連邦政府のSecurity Clearanceの項目から外される法案「MORE Act」が下院議会に紹介されました！\n\nhttps://t.co/kgh9wJI4HN,2019-11-17 04:13 +0000,67968.0,1867.0,0.027469,188.0,1.0,458.0,380.0,372.0,0.0,467.0,0.0,0,0,1,0,0,0,0


In [52]:
df.sort_values('engagements', ascending=False).head(10)

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,user profile clicks,url clicks,hashtag clicks,detail expands,permalink clicks,app opens,app installs,follows,email tweet,dial phone,media views,media engagements
27,1356438150915125250,https://twitter.com/420_note/status/1356438150915125250,今日からオレゴン州で全ての薬物の所持が正式に非犯罪化されました。\n\n下記の量を所持してても刑事罰はありません。\n\n- ヘロイン1g未満\n- MDMAの1g未満または5ピル未満\n- メタンフェタミン2g未満\n- LSDが40枚未満\n- シロシビン12g未満\n- コカイン2g未満\n\nhttps://t.co/NNX5eYZiGv,2021-02-02 03:03 +0000,266019.0,16831.0,0.063270,528.0,6.0,1315.0,1216.0,1595.0,0.0,12112.0,0.0,0,0,2,0,0,57,57
3,1387926835829252097,https://twitter.com/420_note/status/1387926835829252097,"カナダで乾燥大麻を使ったvape をテストするための人員を募集してて、それが年収 50,000カナダドル（約443万円）です。\n\n年間3週間の有給付きで、乾燥大麻を購入すために毎月 250カナダドル（約2.2万円）が給料とは別に支給されます。\n\nhttps://t.co/vHBG3Qgqz7 https://t.co/UBkzFytQyR",2021-04-30 00:28 +0000,86825.0,6329.0,0.072894,178.0,4.0,602.0,258.0,961.0,0.0,2584.0,0.0,0,0,0,0,0,1742,1742
21,1392274960299675655,https://twitter.com/420_note/status/1392274960299675655,ゆうせい荘さんの件が海外メディアに取り上げられました。\n\nhttps://t.co/OuN14T1SCK,2021-05-12 00:26 +0000,58297.0,4446.0,0.076265,105.0,1.0,327.0,223.0,1851.0,0.0,1939.0,0.0,0,0,0,0,0,0,0
80,1225655024664993793,https://twitter.com/420_note/status/1225655024664993793,「大麻で逮捕される世界最悪の地域」の紹介にガッツリ日本が入っています（笑）\n日本の状況は世界基準でも明らかに異常。\n\n「カナダが2018年に合法化した時、日本政府は帰国時に罰則を受ける可能性がある事を示唆し、海外の日本人に大麻と関わらないよう警告する程厳しい」\n\nhttps://t.co/lVsDGqv4PE,2020-02-07 05:38 +0000,44668.0,3986.0,0.089236,117.0,1.0,354.0,254.0,854.0,0.0,2406.0,0.0,0,0,0,0,0,0,0
44,1283780045786329089,https://twitter.com/420_note/status/1283780045786329089,コカインのストリート価格が世界トップクラスに高い日本に38万人のユーザが入れば、麻薬組織にとっては魅力的な市場です。\n\nhttps://t.co/lkzyoKe7jW https://t.co/m9ZgeSYzXb,2020-07-16 15:06 +0000,11914.0,3239.0,0.271865,15.0,0.0,63.0,131.0,78.0,0.0,400.0,0.0,0,0,0,0,0,2552,2552
44,1384300783156350986,https://twitter.com/420_note/status/1384300783156350986,幸せな生活を送る事ができるのであれば基本全ての薬物(カフェイン、酒、大麻等)は一切推奨しません。\n\nしかし、アメリカ在住日本人の80~90%が嗜好・医療大麻合法州で生活し、アメリカ人の91%が嗜好・医療で大麻が合法化されるべきだと思ってる事を考えると #大麻吸ってみたい と思ってもおかしく無い。 https://t.co/uT0iTsIRfm,2021-04-20 00:20 +0000,86219.0,3090.0,0.035839,36.0,0.0,113.0,149.0,28.0,114.0,195.0,0.0,0,0,0,0,0,2455,2455
64,1334134242473771008,https://twitter.com/420_note/status/1334134242473771008,今日の国連の採決で大麻を最も厳しいSchedule IVからSchedule I へ可決されましたが、日本は反対しています。\n\nそれ以外に、大麻チンキをSchedule Iからの除外や、THCが0.2%未満の大麻を麻薬条約から除外する内容など、4つの採決で全て日本は反対しています。。。\n\n詳細は↓\n\nhttps://t.co/PdNlB8NbYp,2020-12-02 13:56 +0000,110146.0,2697.0,0.024486,61.0,4.0,176.0,228.0,883.0,0.0,1345.0,0.0,0,0,0,0,0,0,0
17,1395197162321973251,https://twitter.com/420_note/status/1395197162321973251,#クレイジージャーニー 見ました。全体的には編集に問題があると思いました。「ロサンゼルス州」の誤りでさえ修正しないぐらい時間も予算も無かったのか。\n\n一番致命的なのはデータで物事を語っていない事かなと思いました。以下に記載↓ https://t.co/XEJEgsLbPg,2021-05-20 01:58 +0000,19359.0,2454.0,0.126763,35.0,3.0,114.0,110.0,18.0,32.0,1320.0,0.0,0,0,0,0,0,822,822
13,1198486486074585088,https://twitter.com/420_note/status/1198486486074585088,@takapon_jp 初めまして、サンフランシスコで大麻業界に携わっている日系アメリカ人です。日本語でなるべくわかりやすくこの業界について解説しています。日本で大麻取締法の改正がなぜ進まないかを（長いですが）まとめておりますので、是非ご参考にして頂ければと思います。\n\nhttps://t.co/YRI6jE4UEY,2019-11-24 06:20 +0000,40352.0,2453.0,0.060790,25.0,0.0,89.0,263.0,1957.0,0.0,119.0,0.0,0,0,0,0,0,0,0
40,1284304500702867456,https://twitter.com/420_note/status/1284304500702867456,THCなどを世界で初めて分離した、「大麻研究の父」のイスラエル人の Mechoulam博士が「EPM301」という新たな合成カンナビノイドの開発に成功したと発表。\n\n従来のTHC・CBDより強力で、副作用がほとんど無く吐き気・不安・炎症性疾患の治療になるとのことです。\n\nhttps://t.co/FeFncjagzk,2020-07-18 01:50 +0000,45075.0,2207.0,0.048963,122.0,3.0,464.0,232.0,339.0,0.0,1047.0,0.0,0,0,0,0,0,0,0


In [53]:
df.sort_values('engagements', ascending=False).head(10)['Tweet permalink']

27    https://twitter.com/420_note/status/1356438150915125250
3     https://twitter.com/420_note/status/1387926835829252097
21    https://twitter.com/420_note/status/1392274960299675655
80    https://twitter.com/420_note/status/1225655024664993793
44    https://twitter.com/420_note/status/1283780045786329089
44    https://twitter.com/420_note/status/1384300783156350986
64    https://twitter.com/420_note/status/1334134242473771008
17    https://twitter.com/420_note/status/1395197162321973251
13    https://twitter.com/420_note/status/1198486486074585088
40    https://twitter.com/420_note/status/1284304500702867456
Name: Tweet permalink, dtype: object